# TP : Introduction à la compression JPEG 2000 - CORRIGÉ

**Niveau :** Première NSI

**Objectifs :**
- Comprendre le principe de base de la compression d'images
- Manipuler des calculs de moyennes et différences
- Découvrir la transformation de Haar sur 2 puis 4 points

## Introduction : Pourquoi compresser une image ?

### Le problème

Une image de **816 × 608 pixels** en couleur nécessite :
- 3 octets par pixel (rouge, vert, bleu)
- Soit : 816 × 608 × 3 = **1 488 384 octets** ≈ **1,49 Mo**

Avec la compression JPEG, cette même photo ne pèse que **237 Ko**, soit environ **6,28 fois moins** !

### L'idée principale

Dans une région uniforme (ciel bleu, mur blanc), les pixels voisins ont des valeurs très proches.

Au lieu de stocker chaque pixel individuellement, on peut stocker :
- Leur **moyenne** (qui représente la région)
- Leurs **différences** (qui sont souvent très petites)

Les petites différences peuvent être arrondies à 0 pour **compresser** l'information !

C'est le principe de la **transformation en ondelettes** utilisée par JPEG 2000.

### Simplification pour ce TP

Pour simplifier, nous allons travailler sur une image **en niveaux de gris** (1 seul canal).

En pratique, pour compresser une image couleur :
- On convertit l'image RGB en niveaux de gris, OU
- On applique la transformation séparément sur chaque canal (Rouge, Vert, Bleu)

Dans ce TP, nous nous concentrons sur le principe de base avec des niveaux de gris.

## Importation des bibliothèques

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

## Partie 1 : Transformation sur 2 points

### Le principe

Au lieu de stocker deux nombres `a` et `b`, on peut stocker :
- Leur **moyenne** : $x = \frac{a+b}{2}$
- La **moitié de leur différence** : $y = \frac{b-a}{2}$

On peut retrouver `a` et `b` avec :
- $b = x + y$
- $a = x - y$

**Intérêt** : Si `a` et `b` sont proches, alors `y` sera très petit et pourra être arrondi à 0 pour économiser de l'espace !

### 📝 Exercice 1 : Créer une image de 2 pixels

Créons une image de 2 pixels côte à côte avec PIL.

**Consigne :** Choisissez deux valeurs de niveaux de gris (entre 0 et 255) pour les pixels `a` et `b`.

In [ ]:
# Choix des valeurs des pixels (vous pouvez les modifier)
a = 25
b = 23

# Créer une image PIL de 2 pixels
image_2px = Image.new('L', (2, 1))  # 'L' = niveaux de gris, taille 2x1
image_2px.putpixel((0, 0), a)
image_2px.putpixel((1, 0), b)

# Agrandir l'image pour mieux voir (chaque pixel devient 100x100)
image_2px_grande = image_2px.resize((200, 100), Image.Resampling.NEAREST)

# Affichage
plt.figure(figsize=(8, 3))
plt.imshow(image_2px_grande, cmap='gray', vmin=0, vmax=255)
plt.title(f'Image de 2 pixels : a={a}, b={b}', fontsize=14, fontweight='bold')
plt.xticks([50, 150], ['pixel a', 'pixel b'])
plt.yticks([])
plt.colorbar(label='Niveau de gris')
plt.show()

print(f"Pixels créés : a={a}, b={b}")

### 📝 Exercice 2 : Transformation de Haar sur 2 pixels

Calculez la moyenne `x` et la demi-différence `y` de vos deux pixels.

In [ ]:
# SOLUTION: Calculer la moyenne x
x = (a + b) / 2

# SOLUTION: Calculer la demi-différence y
y = (b - a) / 2

print(f"Transformation de Haar :")
print(f"  Pixels originaux : a={a}, b={b}")
print(f"  Moyenne x = {x}")
print(f"  Demi-différence y = {y}")

### 🔍 Observation

Dans cet exemple :
- La moyenne est `x = 24`
- La demi-différence est `y = -1` (très petit !)

Si on arrondissait `y` à 0, on ne perdrait presque aucune information, mais on pourrait stocker l'information plus efficacement.

### 💾 Le gain de place

**Sans compression :**
- On stocke 2 valeurs : `a = 25` et `b = 23`
- Soit **2 octets** (1 octet par pixel)

**Avec compression :**
- On stocke `x = 24` (la moyenne) : **1 octet**
- On arrondit `y = -1` à `0` (trop petit) : **0 octet**
- Soit **1 octet** au total !

**Gain : 50% d'espace économisé !** 🎉

Bien sûr, on perd un peu de précision (on reconstruira `a = 24` et `b = 24` au lieu de `a = 25` et `b = 23`), mais à l'œil nu, c'est imperceptible !

### 📝 Exercice 3 : Vérification de la reconstruction

Vérifiez qu'on peut retrouver `a` et `b` à partir de `x` et `y`.

In [ ]:
# SOLUTION: Reconstruire a et b à partir de x et y
a_reconstruit = x - y
b_reconstruit = x + y

print(f"Reconstruction :")
print(f"  a reconstruit = {a_reconstruit}")
print(f"  b reconstruit = {b_reconstruit}")
print(f"\nVérification :")
print(f"  a original = {a}")
print(f"  b original = {b}")

## Partie 2 : Transformation sur 4 points (carré 2×2)

### Le principe en 2 dimensions

Pour un carré 2×2 de pixels :

```
a  b
c  d
```

La **transformation de Haar** calcule 4 valeurs :

1. **Rouge (moyenne globale)** : $\frac{a+b+c+d}{4}$ 
   - Représente la couleur globale du carré

2. **Bleu (différences verticales)** : $\frac{(b-a)+(d-c)}{4}$ 
   - Mesure les changements de gauche à droite

3. **Vert (différences horizontales)** : $\frac{(c-a)+(d-b)}{4}$ 
   - Mesure les changements de haut en bas

4. **Violet (différences obliques)** : $\frac{(b+c)-(a+d)}{4}$ 
   - Mesure les changements en diagonale

### 📝 Exercice 4 : Créer une image de 4 pixels (carré 2×2)

Créons maintenant un carré 2×2 avec PIL.

**Consigne :** Choisissez 4 valeurs de niveaux de gris pour les pixels a, b, c, d.

In [ ]:
# Choix des valeurs des 4 pixels (vous pouvez les modifier)
a = 134
b = 224
c = 137
d = 162

# Créer une image PIL de 2×2 pixels
image_4px = Image.new('L', (2, 2))  # 'L' = niveaux de gris, taille 2x2
image_4px.putpixel((0, 0), a)  # Coin haut-gauche
image_4px.putpixel((1, 0), b)  # Coin haut-droit
image_4px.putpixel((0, 1), c)  # Coin bas-gauche
image_4px.putpixel((1, 1), d)  # Coin bas-droit

# Agrandir l'image pour mieux voir (chaque pixel devient 100x100)
image_4px_grande = image_4px.resize((200, 200), Image.Resampling.NEAREST)

# Affichage
plt.figure(figsize=(6, 6))
plt.imshow(image_4px_grande, cmap='gray', vmin=0, vmax=255)
plt.title('Image 2×2 pixels', fontsize=14, fontweight='bold')
plt.xticks([50, 150], ['a, c', 'b, d'])
plt.yticks([50, 150], ['a, b', 'c, d'])

# Ajouter les valeurs sur l'image
positions = [(50, 50, a, 'a'), (150, 50, b, 'b'), 
             (50, 150, c, 'c'), (150, 150, d, 'd')]
for x, y, val, nom in positions:
    plt.text(x, y, f'{nom}={val}', ha='center', va='center', 
            color='red', fontsize=16, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.colorbar(label='Niveau de gris')
plt.show()

print(f"Carré 2×2 créé :")
print(f"  [{a:3d}  {b:3d}]")
print(f"  [{c:3d}  {d:3d}]")

### 📝 Exercice 5 : Implémenter la transformation de Haar 2×2

In [ ]:
# SOLUTION: Calculer les 4 coefficients à partir de a, b, c, d
rouge = (a + b + c + d) / 4      # Moyenne globale
bleu = ((b - a) + (d - c)) / 4   # Différences verticales
vert = ((c - a) + (d - b)) / 4   # Différences horizontales
violet = ((b + c) - (a + d)) / 4 # Différences obliques

print("Transformation de Haar 2×2 :")
print(f"\nCarré original :")
print(f"  [{a:3d}  {b:3d}]")
print(f"  [{c:3d}  {d:3d}]")
print(f"\nCarré transformé :")
print(f"  [ROUGE={rouge:.1f}  BLEU={bleu:.1f}]")
print(f"  [VERT={vert:.1f}   VIOLET={violet:.1f}]")

### 🔍 Analyse des résultats

Résultats obtenus :
- **Rouge = 164.25** : c'est la couleur moyenne du carré
- **Bleu = 26.25** : différence importante (changement vertical)
- **Vert = -15.75** : différence moyenne (changement horizontal)
- **Violet = 10.75** : petite différence diagonale

**Interprétation** : 
- La valeur rouge contient l'information principale
- Les trois autres valeurs (bleu, vert, violet) représentent les détails
- Dans une zone uniforme, ces trois valeurs seraient proches de 0 !

### 💾 Le gain de place expliqué

**Sans compression :**
- On stocke 4 valeurs : `a=134, b=224, c=137, d=162`
- Soit **4 octets** (1 octet par pixel)

**Avec compression :**
- **Rouge = 164.25** (important) : on stocke sur **1 octet**
- **Bleu = 26.25** (moyen) : on peut coder sur **quelques bits** au lieu d'1 octet
- **Vert = -15.75** (moyen) : quelques bits aussi
- **Violet = 10.75** (petit) : quelques bits

Au lieu de 4 octets, on peut stocker sur environ **2 octets** !

**Gain : 50% d'espace économisé !**

### 📊 Cas idéal : zone uniforme

Testons avec un carré où tous les pixels sont identiques (par exemple `a=b=c=d=100`) :
- Rouge = 100 (moyenne)
- Bleu = 0 (pas de changement vertical)
- Vert = 0 (pas de changement horizontal)
- Violet = 0 (pas de changement diagonal)

Dans ce cas, on stocke seulement **1 valeur** au lieu de 4 !

**Gain : 75% d'espace économisé !** 🎉🎉

### 📊 Visualisation graphique : avant et après transformation

In [ ]:
# Visualisation du carré 2×2 avec PIL
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# === AVANT : Carré original ===
axes[0].imshow(image_4px_grande, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('AVANT : Carré original 2×2', fontsize=14, fontweight='bold')
axes[0].set_xticks([50, 150])
axes[0].set_yticks([50, 150])
axes[0].set_xticklabels(['colonne 0', 'colonne 1'])
axes[0].set_yticklabels(['ligne 0', 'ligne 1'])
axes[0].axis('on')

# Ajouter les valeurs sur l'image
positions = [(50, 50, a, 'a'), (150, 50, b, 'b'), 
             (50, 150, c, 'c'), (150, 150, d, 'd')]
for x, y, val, nom in positions:
    axes[0].text(x, y, f'{nom}={val}', ha='center', va='center', 
                color='red', fontsize=18, fontweight='bold')

# === APRÈS : Carré transformé ===
# Créer une image colorée pour mieux visualiser les différents coefficients
img_transfo = Image.new('RGB', (200, 200))
draw = ImageDraw.Draw(img_transfo)

# Définir les couleurs pour chaque quadrant
couleurs = [
    [(200, 50, 50), (50, 50, 200)],   # Rouge, Bleu
    [(50, 200, 50), (150, 50, 150)]   # Vert, Violet
]

labels = [
    ['ROUGE\n(moyenne)', 'BLEU\n(vertical)'],
    ['VERT\n(horizontal)', 'VIOLET\n(oblique)']
]

valeurs = [[rouge, bleu], [vert, violet]]

# Dessiner les 4 quadrants colorés
for i in range(2):
    for j in range(2):
        x0, y0 = j * 100, i * 100
        draw.rectangle([x0, y0, x0 + 100, y0 + 100], fill=couleurs[i][j])

axes[1].imshow(img_transfo)
axes[1].set_title('APRÈS : Transformation de Haar', fontsize=14, fontweight='bold')
axes[1].set_xticks([50, 150])
axes[1].set_yticks([50, 150])
axes[1].set_xticklabels(['quadrant 1', 'quadrant 2'])
axes[1].set_yticklabels(['quadrant 1', 'quadrant 2'])
axes[1].axis('on')

# Ajouter les labels et valeurs
for i in range(2):
    for j in range(2):
        axes[1].text(j*100 + 50, i*100 + 50, 
                    f'{labels[i][j]}\n{valeurs[i][j]:.1f}', 
                    ha='center', va='center', color='white', 
                    fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("Interprétation :")
print(f"  • ROUGE ({rouge:.1f}) = valeur moyenne du carré")
print(f"  • BLEU ({bleu:.1f}) = changements verticaux (gauche -> droite)")
print(f"  • VERT ({vert:.1f}) = changements horizontaux (haut -> bas)")
print(f"  • VIOLET ({violet:.1f}) = changements en diagonale")

### 📝 Exercice 6 : Test avec un carré uniforme

Pour bien comprendre le gain de compression, testons avec un carré où tous les pixels ont la même valeur !

In [ ]:
# Carré uniforme (tous les pixels à 100)
a_uni = 100
b_uni = 100
c_uni = 100
d_uni = 100

# Calcul de la transformation
rouge_uni = (a_uni + b_uni + c_uni + d_uni) / 4
bleu_uni = ((b_uni - a_uni) + (d_uni - c_uni)) / 4
vert_uni = ((c_uni - a_uni) + (d_uni - b_uni)) / 4
violet_uni = ((b_uni + c_uni) - (a_uni + d_uni)) / 4

print("Carré uniforme :")
print(f"  [{a_uni:3d}  {b_uni:3d}]")
print(f"  [{c_uni:3d}  {d_uni:3d}]")
print(f"\nTransformation :")
print(f"  [ROUGE={rouge_uni:.1f}  BLEU={bleu_uni:.1f}]")
print(f"  [VERT={vert_uni:.1f}   VIOLET={violet_uni:.1f}]")
print(f"\n>>> On remarque que toutes les différences sont à 0 !")
print(f"    Seule la moyenne (100) contient de l'information.")
print(f"    C'est parfait pour la compression !")

## Conclusion

### Ce que nous avons appris

1. **Principe de base** : On peut remplacer des valeurs par leur moyenne et leurs différences

2. **Intérêt pour la compression** :
   - Les moyennes contiennent l'information principale
   - Les différences sont souvent petites dans les zones uniformes
   - On peut arrondir les petites différences à 0 pour compresser

3. **Transformation de Haar** :
   - Sur 2 points : on obtient 1 moyenne + 1 différence
   - Sur 4 points (2×2) : on obtient 1 moyenne + 3 différences (verticale, horizontale, diagonale)

### 💾 Le principe de la compression

**L'astuce magique :**
1. On transforme les pixels en moyennes + différences
2. Les différences sont souvent très petites (proches de 0)
3. On peut :
   - Arrondir les petites valeurs à 0 (compression avec perte)
   - Coder les petites valeurs sur moins de bits (moins d'espace)
   - Ne pas stocker les valeurs nulles (encore moins d'espace !)

**Résultat :**
- Dans une zone uniforme : **jusqu'à 75% d'économie d'espace !**
- Dans une zone détaillée : **environ 30-50% d'économie**
- Sur une image complète : **facteur 6 de compression en moyenne**

C'est pour ça qu'une photo de 1,5 Mo peut tenir dans 250 Ko ! 🎉

### Et après ?

Pour compresser une vraie image :
- On divise l'image en petits carrés 2×2
- On applique la transformation sur chaque carré
- On peut même réappliquer la transformation plusieurs fois (sur les moyennes)
- On arrondit ou code efficacement les petites valeurs

C'est le principe du **JPEG 2000**, qui utilise des ondelettes plus sophistiquées que celle de Haar !

## 🎯 Questions de réflexion - RÉPONSES

1. **Pourquoi la transformation en moyennes et différences est-elle utile pour la compression ?**

   *Réponse :* Dans les zones uniformes d'une image, les pixels voisins ont des valeurs très proches. La transformation produit alors de grandes moyennes (qui représentent la structure) et de petites différences (qui peuvent être arrondies à 0 pour compresser).

2. **Dans quel cas les différences (bleu, vert, violet) seront-elles proches de 0 ?**

   *Réponse :* Quand les pixels sont tous de la même couleur ou presque (zone uniforme : ciel, mur, etc.).

3. **Quel type d'image se compresse le mieux : une photo d'un ciel bleu uniforme ou une photo d'une foule de personnes ? Pourquoi ?**

   *Réponse :* Le ciel bleu uniforme se compresse beaucoup mieux ! Il contient de grandes zones uniformes qui donnent des différences proches de 0. Une foule contient beaucoup de détails différents, donc les différences restent importantes.

4. **À votre avis, peut-on reconstruire exactement l'image originale à partir de la transformation ? Pourquoi ?**

   *Réponse :* Oui, si on ne fait que la transformation (sans arrondir). Les formules mathématiques sont réversibles. Mais si on arrondit les petites valeurs à 0 pour compresser, on perd un peu d'information (compression avec perte).

---

**Source :** D'après un article de Christiane Rousseau, Université de Montréal  
**Adapté pour 1ère NSI - Version simplifiée - CORRIGÉ**